In [ ]:
using Revise
using BSON
using DataDeps, MAT, MLUtils

Replace `_bcks_` with `_ppo_` or `_sac_` to preprocess all collected data. Traverse and preprocess all collected `mat` data as follows with proper thresholds.

In [ ]:

T = Float32
file = matopen("./data_bcks_parabolic_train1.mat")
    
x_data = T.(collect(read(file, "a")))
y_data = T.(collect(read(file, "u")))
close(file)

file = matopen("./data_bcks_parabolic_train2.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

file = matopen("./data_bcks_parabolic_train3.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

file = matopen("./data_bcks_parabolic_test.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)


@show size(x_data)
@show size(y_data)


threshold = -0.1
# pf: 1, not pf: 0
pf_labels = ones(size(x_data))
# safe: 1, not safe: 0
safe_labels = -ones(size(x_data))
not_pf = 0
pf_training = 0
for i in 1:size(x_data, 1)
    for j in 1:size(x_data, 2)
        if y_data[i,j]-0.0 < threshold
            safe_labels[i,j] = 0
        end
    end
    for j in 1:size(x_data, 2)
        if safe_labels[i,end+1-j] == -1
            safe_labels[i,end+1-j] = 1
        else
            break
        end
    end
    if y_data[i,end]-0.0 < threshold
        pf_labels[i, :] .= 0
        not_pf += 1
    else
        # pf and used for training
        pf_labels[i, :] .= 1
        pf_training += 1
    end
end

matwrite("data_bcks_parabolic_ge-0.1safe.mat", Dict(
	"a" => x_data,
	"u" => y_data,
    "pf" => pf_labels,
    "safe" => safe_labels
))
@show not_pf, pf_training, 50000-not_pf-pf_training

In [ ]:

T = Float32
file = matopen("./data_sac_parabolic_train1.mat")
    
x_data = T.(collect(read(file, "a")))
y_data = T.(collect(read(file, "u")))
close(file)

file = matopen("./data_sac_parabolic_train2.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

file = matopen("./data_sac_parabolic_train3.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)


file = matopen("./data_sac_parabolic_test.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)


@show size(x_data)
@show size(y_data)


threshold = 0.2
# pf: 1, not pf: 0
pf_labels = ones(size(x_data))
# safe: 1, not safe: 0
safe_labels = -ones(size(x_data))
not_pf = 0
pf_training = 0
for i in 1:size(x_data, 1)
    for j in 1:size(x_data, 2)
        if y_data[i,j]-0.0 > threshold
            safe_labels[i,j] = 0
        end
    end
    for j in 1:size(x_data, 2)
        if safe_labels[i,end+1-j] == -1
            safe_labels[i,end+1-j] = 1
        else
            break
        end
    end
    if y_data[i,end]-0.0 > threshold
        pf_labels[i, :] .= 0
        not_pf += 1
    else
        # pf and used for training
        pf_labels[i, :] .= 1
        pf_training += 1
    end
end

matwrite("data_sac_parabolic_le0.2safe.mat", Dict(
	"a" => x_data,
	"u" => y_data,
    "pf" => pf_labels,
    "safe" => safe_labels
))
@show not_pf, pf_training, 50000-not_pf-pf_training

In [ ]:

T = Float32

file = matopen("./data_sac_parabolic_test_dense.mat")
    

x_data = T.(collect(read(file, "a")))
y_data = T.(collect(read(file, "u")))
close(file)

@show size(x_data)
@show size(y_data)


threshold = 0.43
# pf: 1, not pf: 0
pf_labels = ones(size(x_data))
# safe: 1, not safe: 0
safe_labels = -ones(size(x_data))
not_pf = 0
pf_training = 0
for i in 1:size(x_data, 1)
    for j in 1:size(x_data, 2)
        if abs(y_data[i,j]-0.17) > threshold
            safe_labels[i,j] = 0
        end
    end
    for j in 1:size(x_data, 2)
        if safe_labels[i,end+1-j] == -1
            safe_labels[i,end+1-j] = 1
        else
            break
        end
    end
    if abs(y_data[i,end]-0.17) > threshold
        pf_labels[i, :] .= 0
        not_pf += 1
    else
        # pf and used for training
        pf_labels[i, :] .= 1
        pf_training += 1
    end
end

matwrite("data_sac_parabolic_abs_test_dense.mat", Dict(
	"a" => x_data,
	"u" => y_data,
    "pf" => pf_labels,
    "safe" => safe_labels
))
@show not_pf, pf_training, 5000-not_pf-pf_training